In [1]:
import tensorflow as tf
import datetime
import numpy as np

import data_tool as dt
import keras_tool as kt

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Clear any logs from previous runs
%rm -rf ./temp6/logs/

In [3]:
user_number = 100
code_length = 50
epoch_number = 30
batch_size = 1000

In [4]:
estimator = keras.Sequential(
    [
        tf.keras.layers.BatchNormalization(name="bp1"),
        layers.Dense(user_number + code_length, activation="relu", name="layer1"),
        tf.keras.layers.BatchNormalization(name="bp2"),
        layers.Dense(user_number + code_length, activation="relu", name="layer2"),
        tf.keras.layers.BatchNormalization(name="bp3"),
        layers.Dense(user_number + code_length, activation="relu", name="layer3"),
        tf.keras.layers.BatchNormalization(name="pb4"),
        layers.Dense(user_number, activation="relu", name="layer4"),
    ],name = 'estimator'
)
# Call model on a test input
y = tf.ones((3, code_length))
x_hat = estimator(y)
estimator.summary()

signature_matrix = dt.get_random_binary_signature_matrix(user_number, code_length)
y_set, x_set, h_set = dt.get_dataset(1000000, signature_matrix, is_fading=1)
y_set = y_set.astype("float32")
x_set = x_set.astype("float32")
h_set = h_set.astype("float32")
y_test, x_test, h_test = dt.get_dataset(10000, signature_matrix, is_fading=1)
y_test = y_test.astype("float32")
x_test = x_test.astype("float32")
h_test = h_test.astype("float32")

Model: "estimator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bp1 (BatchNormalization)     (3, 50)                   200       
_________________________________________________________________
layer1 (Dense)               (3, 150)                  7650      
_________________________________________________________________
bp2 (BatchNormalization)     (3, 150)                  600       
_________________________________________________________________
layer2 (Dense)               (3, 150)                  22650     
_________________________________________________________________
bp3 (BatchNormalization)     (3, 150)                  600       
_________________________________________________________________
layer3 (Dense)               (3, 150)                  22650     
_________________________________________________________________
pb4 (BatchNormalization)     (3, 150)                  60

In [5]:
estimator.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[kt.NMSE_Accuracy(), kt.Bit_Error_Ratio_Threshold_0_01(), kt.Block_Error_Ratio_Threshold_0_01(), "Accuracy"],
)

log_dir="./temp6/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = estimator.fit(y_set, h_set, batch_size=1000, epochs=30, validation_split=0.2, callbacks=[tensorboard_callback])

test_scores = estimator.evaluate(y_test, h_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1:])

Epoch 1/30
800/800 [==============================] - 14s 14ms/step - loss: 0.1084 - nmse_accuracy: -3.2615 - bit_error_ratio: 0.2283 - block_error_ratio: 0.9902 - accuracy: 0.6901 - val_loss: 0.0179 - val_nmse_accuracy: -10.9990 - val_bit_error_ratio: 0.0698 - val_block_error_ratio: 0.9473 - val_accuracy: 0.8383
Epoch 2/30
800/800 [==============================] - 13s 16ms/step - loss: 0.0139 - nmse_accuracy: -12.2585 - bit_error_ratio: 0.0633 - block_error_ratio: 0.9122 - accuracy: 0.8430 - val_loss: 0.0105 - val_nmse_accuracy: -13.5039 - val_bit_error_ratio: 0.0598 - val_block_error_ratio: 0.8934 - val_accuracy: 0.8444
Epoch 3/30
800/800 [==============================] - 12s 15ms/step - loss: 0.0100 - nmse_accuracy: -13.7290 - bit_error_ratio: 0.0561 - block_error_ratio: 0.8808 - accuracy: 0.8481 - val_loss: 0.0092 - val_nmse_accuracy: -14.1325 - val_bit_error_ratio: 0.0572 - val_block_error_ratio: 0.8788 - val_accuracy: 0.8461
Epoch 4/30
800/800 [==============================] -

Epoch 27/30
800/800 [==============================] - 10s 13ms/step - loss: 0.0068 - nmse_accuracy: -15.4763 - bit_error_ratio: 0.0512 - block_error_ratio: 0.8481 - accuracy: 0.8500 - val_loss: 0.0066 - val_nmse_accuracy: -15.5937 - val_bit_error_ratio: 0.0503 - val_block_error_ratio: 0.8332 - val_accuracy: 0.8508
Epoch 28/30
800/800 [==============================] - 10s 13ms/step - loss: 0.0067 - nmse_accuracy: -15.5409 - bit_error_ratio: 0.0510 - block_error_ratio: 0.8458 - accuracy: 0.8501 - val_loss: 0.0067 - val_nmse_accuracy: -15.5761 - val_bit_error_ratio: 0.0522 - val_block_error_ratio: 0.8409 - val_accuracy: 0.8487
Epoch 29/30
800/800 [==============================] - 12s 15ms/step - loss: 0.0067 - nmse_accuracy: -15.5020 - bit_error_ratio: 0.0510 - block_error_ratio: 0.8457 - accuracy: 0.8501 - val_loss: 0.0065 - val_nmse_accuracy: -15.6649 - val_bit_error_ratio: 0.0495 - val_block_error_ratio: 0.8276 - val_accuracy: 0.8517
Epoch 30/30
800/800 [============================

In [6]:
y_set_10dB, x_set_10dB, h_set_10dB, y_set_out_10dB = dt.get_dataset_with_noise(1000000, signature_matrix, 10, is_fading=1)
y_set_10dB = y_set_10dB.astype("float32")
x_set_10dB = x_set_10dB.astype("float32")
h_set_10dB = h_set_10dB.astype("float32")
y_set_out_10dB = y_set_out_10dB.astype("float32")
y_test_10dB, x_test_10dB, h_test_10dB, y_test_out_10dB = dt.get_dataset_with_noise(10000, signature_matrix, 10, is_fading=1)
y_test_10dB = y_test_10dB.astype("float32")
x_test_10dB = x_test_10dB.astype("float32")
h_test_10dB = h_test_10dB.astype("float32")
y_test_out_10dB = y_test_out_10dB.astype("float32")

In [7]:
print(estimator.evaluate(y_test_10dB, h_test_10dB, verbose=2))
print(estimator.evaluate(y_test_out_10dB, h_test_10dB, verbose=2))

313/313 - 0s - loss: 0.0235 - nmse_accuracy: -8.7575e+00 - bit_error_ratio: 0.1653 - block_error_ratio: 1.0000 - accuracy: 0.7360
[0.02350356988608837, -8.757534980773926, 0.16532500088214874, 1.0, 0.7359650135040283]
313/313 - 1s - loss: 0.0064 - nmse_accuracy: -1.5636e+01 - bit_error_ratio: 0.0487 - block_error_ratio: 0.8186 - accuracy: 0.8525
[0.006403318606317043, -15.636138916015625, 0.048650000244379044, 0.8185999989509583, 0.852528989315033]


In [8]:
# 313/313 - 0s - loss: 0.0238 - nmse_accuracy: -8.7320e+00 - bit_error_ratio: 0.1691 - block_error_ratio: 1.0001 - accuracy: 0.7320
# [0.023814434185624123, -8.732009887695312, 0.16908100247383118, 1.000100016593933, 0.732043981552124]
# 313/313 - 0s - loss: 0.0061 - nmse_accuracy: -1.5955e+01 - bit_error_ratio: 0.0474 - block_error_ratio: 0.8087 - accuracy: 0.8537
# [0.006111446768045425, -15.95509147644043, 0.04741999879479408, 0.8086808919906616, 0.853672981262207]

In [9]:
estimator_with_10dB = keras.Sequential(
    [
        tf.keras.layers.BatchNormalization(name="bp1"),
        layers.Dense(user_number + code_length, activation="relu", name="layer1"),
        tf.keras.layers.BatchNormalization(name="bp2"),
        layers.Dense(user_number + code_length, activation="relu", name="layer2"),
        tf.keras.layers.BatchNormalization(name="bp3"),
        layers.Dense(user_number + code_length, activation="relu", name="layer3"),
        tf.keras.layers.BatchNormalization(name="pb4"),
        layers.Dense(user_number, activation="relu", name="layer4"),
    ],name = 'estimator_with_10dB'
)
# Call model on a test input
y = tf.ones((3, code_length))
x_hat = estimator_with_10dB(y)
estimator_with_10dB.summary()

Model: "estimator_with_10dB"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bp1 (BatchNormalization)     (3, 50)                   200       
_________________________________________________________________
layer1 (Dense)               (3, 150)                  7650      
_________________________________________________________________
bp2 (BatchNormalization)     (3, 150)                  600       
_________________________________________________________________
layer2 (Dense)               (3, 150)                  22650     
_________________________________________________________________
bp3 (BatchNormalization)     (3, 150)                  600       
_________________________________________________________________
layer3 (Dense)               (3, 150)                  22650     
_________________________________________________________________
pb4 (BatchNormalization)     (3, 150)          

In [10]:
estimator_with_10dB.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[kt.NMSE_Accuracy(), kt.Bit_Error_Ratio_Threshold_0_01(), kt.Block_Error_Ratio_Threshold_0_01(), "Accuracy"],
)

log_dir="./temp6/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = estimator_with_10dB.fit(y_set_10dB, h_set_10dB, batch_size=1000, epochs=30, validation_split=0.2, callbacks=[tensorboard_callback])

test_scores = estimator_with_10dB.evaluate(y_test_10dB, h_test_10dB, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1:])

Epoch 1/30
800/800 [==============================] - 13s 15ms/step - loss: 0.1156 - nmse_accuracy: -2.7871 - bit_error_ratio: 0.2396 - block_error_ratio: 0.9943 - accuracy: 0.6807 - val_loss: 0.0310 - val_nmse_accuracy: -8.2877 - val_bit_error_ratio: 0.0856 - val_block_error_ratio: 0.9732 - val_accuracy: 0.8284
Epoch 2/30
800/800 [==============================] - 13s 16ms/step - loss: 0.0256 - nmse_accuracy: -9.0938 - bit_error_ratio: 0.0826 - block_error_ratio: 0.9736 - accuracy: 0.8292 - val_loss: 0.0212 - val_nmse_accuracy: -9.9119 - val_bit_error_ratio: 0.0782 - val_block_error_ratio: 0.9734 - val_accuracy: 0.8317
Epoch 3/30
800/800 [==============================] - 10s 12ms/step - loss: 0.0207 - nmse_accuracy: -10.0184 - bit_error_ratio: 0.0755 - block_error_ratio: 0.9675 - accuracy: 0.8345 - val_loss: 0.0197 - val_nmse_accuracy: -10.2253 - val_bit_error_ratio: 0.0762 - val_block_error_ratio: 0.9725 - val_accuracy: 0.8332
Epoch 4/30
800/800 [==============================] - 11

Epoch 27/30
800/800 [==============================] - 9s 12ms/step - loss: 0.0167 - nmse_accuracy: -10.8960 - bit_error_ratio: 0.0745 - block_error_ratio: 0.9770 - accuracy: 0.8327 - val_loss: 0.0165 - val_nmse_accuracy: -10.9592 - val_bit_error_ratio: 0.0732 - val_block_error_ratio: 0.9757 - val_accuracy: 0.8341
Epoch 28/30
800/800 [==============================] - 10s 12ms/step - loss: 0.0166 - nmse_accuracy: -10.8995 - bit_error_ratio: 0.0745 - block_error_ratio: 0.9775 - accuracy: 0.8327 - val_loss: 0.0165 - val_nmse_accuracy: -10.9608 - val_bit_error_ratio: 0.0732 - val_block_error_ratio: 0.9756 - val_accuracy: 0.8340
Epoch 29/30
800/800 [==============================] - 10s 13ms/step - loss: 0.0166 - nmse_accuracy: -10.8902 - bit_error_ratio: 0.0745 - block_error_ratio: 0.9776 - accuracy: 0.8326 - val_loss: 0.0166 - val_nmse_accuracy: -10.9368 - val_bit_error_ratio: 0.0755 - val_block_error_ratio: 0.9793 - val_accuracy: 0.8316
Epoch 30/30
800/800 [=============================

In [12]:
test_scores = estimator_with_10dB.evaluate(y_test, h_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1:])

313/313 - 1s - loss: 0.0088 - nmse_accuracy: -1.3985e+01 - bit_error_ratio: 0.0296 - block_error_ratio: 0.6975 - accuracy: 0.8774
Test loss: 0.00882832333445549
Test accuracy: [-13.984776496887207, 0.02962600067257881, 0.6974999904632568, 0.8773810267448425]


In [11]:
# %tensorboard --logdir ./temp6/logs/fit